In [1]:
sc

<SparkContext master=local[*] appName=PySparkShell>

In [2]:
spark

In [1]:
spark.stop()

In [2]:
from pyspark.sql import SparkSession
myssql_connector_path = "/home/hadoop/Downloads/mysql-connector-java-8.0.13.jar"

In [3]:
spark = SparkSession.builder.appName("ETL Pipeline").getOrCreate()

In [4]:
hremployeeDF = spark.read.format("jdbc")\
.option("url","jdbc:mysql://localhost:3306/hremployeeDB")\
.option("dbtable","hremployee").option("user","root").option("password","hadoop@123")\
.option("driver", "com.mysql.cj.jdbc.Driver").load()

In [5]:
hremployeeDF.show()

+----------+--------------------+--------------------+---------+------+---+-------------+-------------+--------------+-----------------+--------------+--------+---------------+----------+------+----------+--------+------+-----------------------+---------------+---------------------+---------------+------------------+
|EmployeeID|          Department|             JobRole|Attrition|Gender|Age|MaritalStatus|    Education|EducationField|   BusinessTravel|JobInvolvement|JobLevel|JobSatisfaction|Hourlyrate|Income|Salaryhike|OverTime|Workex|YearsSinceLastPromotion|EmpSatisfaction|TrainingTimesLastYear|WorkLifeBalance|Performance_Rating|
+----------+--------------------+--------------------+---------+------+---+-------------+-------------+--------------+-----------------+--------------+--------+---------------+----------+------+----------+--------+------+-----------------------+---------------+---------------------+---------------+------------------+
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
|
+--

In [7]:
hremployeeDF.createOrReplaceTempView('hremployee')

In [8]:
hremployeeDF.explain()

== Physical Plan ==
*(1) Scan JDBCRelation(hremployee) [numPartitions=1] [EmployeeID#0,Department#1,JobRole#2,Attrition#3,Gender#4,Age#5,MaritalStatus#6,Education#7,EducationField#8,BusinessTravel#9,JobInvolvement#10,JobLevel#11,JobSatisfaction#12,Hourlyrate#13,Income#14,Salaryhike#15,OverTime#16,Workex#17,YearsSinceLastPromotion#18,EmpSatisfaction#19,TrainingTimesLastYear#20,WorkLifeBalance#21,Performance_Rating#22] PushedFilters: [], ReadSchema: struct<EmployeeID:string,Department:string,JobRole:string,Attrition:string,Gender:string,Age:int,...


### Display no of rows and no of cols
           * show the number of records and number of column

In [16]:
spark.sql("""
select count(*) as ROW_COUNT, (select count(*) from hremployee.columns) as COL_COUNT from hremployee
""").show()

AnalysisException: "Table or view not found: `hremployee`.`columns`; line 2 pos 52;\n'Aggregate [count(1) AS ROW_COUNT#108L, scalar-subquery#109 [] AS COL_COUNT#110]\n:  +- 'Aggregate [unresolvedalias(count(1), None)]\n:     +- 'UnresolvedRelation `hremployee`.`columns`\n+- SubqueryAlias `hremployee`\n   +- Relation[EmployeeID#10,Department#11,JobRole#12,Attrition#13,Gender#14,Age#15,MaritalStatus#16,Education#17,EducationField#18,BusinessTravel#19,JobInvolvement#20,JobLevel#21,JobSatisfaction#22,Hourlyrate#23,Income#24,Salaryhike#25,OverTime#26,Workex#27,YearsSinceLastPromotion#28,EmpSatisfaction#29,TrainingTimesLastYear#30,WorkLifeBalance#31,Performance_Rating#32] csv\n"

### 2. Write a Query to show first three employee from each Job Role

In [9]:
spark.sql("""
select EmployeeID, Department, JobRole, row_number() over(partition by JobRole order by EmployeeID) 
as row_num from hremployee""").show()

+----------+----------+---------------+-------+
|EmployeeID|Department|        JobRole|row_num|
+----------+----------+---------------+-------+
|         1|     Sales|Sales Executive|      1|
|      1012|     Sales|Sales Executive|      2|
|      1014|     Sales|Sales Executive|      3|
|      1020|     Sales|Sales Executive|      4|
|      1026|     Sales|Sales Executive|      5|
|      1027|     Sales|Sales Executive|      6|
|      1031|     Sales|Sales Executive|      7|
|      1032|     Sales|Sales Executive|      8|
|      1039|     Sales|Sales Executive|      9|
|      1042|     Sales|Sales Executive|     10|
|      1048|     Sales|Sales Executive|     11|
|      1049|     Sales|Sales Executive|     12|
|      1050|     Sales|Sales Executive|     13|
|      1052|     Sales|Sales Executive|     14|
|      1058|     Sales|Sales Executive|     15|
|      1059|     Sales|Sales Executive|     16|
|      1063|     Sales|Sales Executive|     17|
|      1067|     Sales|Sales Executive| 

In [10]:
spark.sql("""select * from
(select EmployeeID, Department, JobRole, row_number() over(partition by JobRole order by EmployeeID) 
as row_num from hremployee) where row_num < 4
""").show()

+----------+--------------------+--------------------+-------+
|EmployeeID|          Department|             JobRole|row_num|
+----------+--------------------+--------------------+-------+
|         1|               Sales|     Sales Executive|      1|
|      1012|               Sales|     Sales Executive|      2|
|      1014|               Sales|     Sales Executive|      3|
|      1003|Research & Develo...|Manufacturing Dir...|      1|
|      1034|Research & Develo...|Manufacturing Dir...|      2|
|      1038|Research & Develo...|Manufacturing Dir...|      3|
|       100|Research & Develo...|Laboratory Techni...|      1|
|      1001|Research & Develo...|Laboratory Techni...|      2|
|      1002|Research & Develo...|Laboratory Techni...|      3|
|      1013|               Sales|Sales Representative|      1|
|      1022|               Sales|Sales Representative|      2|
|      1057|               Sales|Sales Representative|      3|
|        10|Research & Develo...|Healthcare Repres...| 

### 3. Write a query to show top three employee from each jobrole earning maxmimum salary

In [19]:
hremployeeDF.printSchema()

root
 |-- EmployeeID: string (nullable = true)
 |-- Department: string (nullable = true)
 |-- JobRole: string (nullable = true)
 |-- Attrition: string (nullable = true)
 |-- Gender: string (nullable = true)
 |-- Age: integer (nullable = true)
 |-- MaritalStatus: string (nullable = true)
 |-- Education: string (nullable = true)
 |-- EducationField: string (nullable = true)
 |-- BusinessTravel: string (nullable = true)
 |-- JobInvolvement: string (nullable = true)
 |-- JobLevel: integer (nullable = true)
 |-- JobSatisfaction: string (nullable = true)
 |-- Hourlyrate: integer (nullable = true)
 |-- Income: integer (nullable = true)
 |-- Salaryhike: integer (nullable = true)
 |-- OverTime: string (nullable = true)
 |-- Workex: integer (nullable = true)
 |-- YearsSinceLastPromotion: integer (nullable = true)
 |-- EmpSatisfaction: string (nullable = true)
 |-- TrainingTimesLastYear: integer (nullable = true)
 |-- WorkLifeBalance: string (nullable = true)
 |-- Performance_Rating: string (null

In [23]:
spark.sql("""
select * from
(select EmployeeID, Income, JobRole, RANK() over(partition by JobRole order by Income desc) 
as rank from hremployee) where rank < 4
""").show()

+----------+------+--------------------+----+
|EmployeeID|Income|             JobRole|rank|
+----------+------+--------------------+----+
|        99| 13872|     Sales Executive|   1|
|       545| 13770|     Sales Executive|   2|
|       839| 13758|     Sales Executive|   3|
|       722| 13973|Manufacturing Dir...|   1|
|       628| 13826|Manufacturing Dir...|   2|
|       744| 13726|Manufacturing Dir...|   3|
|       678|  7403|Laboratory Techni...|   1|
|       817|  6782|Laboratory Techni...|   2|
|       945|  6674|Laboratory Techni...|   3|
|       565|  6632|Sales Representative|   1|
|      1308|  5405|Sales Representative|   2|
|      1220|  4502|Sales Representative|   3|
|      1181| 13966|Healthcare Repres...|   1|
|       317| 13964|Healthcare Repres...|   2|
|       190| 13734|Healthcare Repres...|   3|
|        68|  9724|  Research Scientist|   1|
|      1315|  6962|  Research Scientist|   2|
|      1305|  6854|  Research Scientist|   3|
|       191| 19999|             Ma

### 4) Show top 3 highest package for each job role

In [26]:
spark.sql("select EmployeeID, Income, JobRole,Department from hremployee order by Income Desc Limit 3").show()

+----------+------+-----------------+--------------------+
|EmployeeID|Income|          JobRole|          Department|
+----------+------+-----------------+--------------------+
|       191| 19999|          Manager|Research & Develo...|
|       747| 19973|Research Director|Research & Develo...|
|       852| 19943|          Manager|Research & Develo...|
+----------+------+-----------------+--------------------+



#### 5.Dense_Rank()

In [27]:
spark.sql("""
select employeeid,department,jobrole, age, gender, income, dense_rank()
over(partition by jobrole order by age desc) as Rank_By_Age
from hremployee
""").show()

+----------+----------+---------------+---+------+------+-----------+
|employeeid|department|        jobrole|age|gender|income|Rank_By_Age|
+----------+----------+---------------+---+------+------+-----------+
|       428|     Sales|Sales Executive| 60|Female| 10266|          1|
|       537|     Sales|Sales Executive| 60|  Male|  5405|          1|
|       880|     Sales|Sales Executive| 60|  Male|  5220|          1|
|        64|     Sales|Sales Executive| 59|Female|  7637|          2|
|        71|     Sales|Sales Executive| 59|Female|  5473|          2|
|       898|     Sales|Sales Executive| 59|Female|  5171|          2|
|        99|     Sales|Sales Executive| 58|  Male| 13872|          3|
|       402|     Sales|Sales Executive| 56|Female| 13212|          4|
|      1371|     Sales|Sales Executive| 56|Female|  5380|          4|
|        83|     Sales|Sales Executive| 55|  Male| 10239|          5|
|       609|     Sales|Sales Executive| 55|  Male|  5160|          5|
|       976|     Sal

In [28]:
spark.sql("""
select employeeid,department,jobrole, age, gender, income,workex, dense_rank()
over(partition by jobrole order by age desc, workex desc) as Rank_By_Age
from hremployee
""").show()

+----------+----------+---------------+---+------+------+------+-----------+
|employeeid|department|        jobrole|age|gender|income|workex|Rank_By_Age|
+----------+----------+---------------+---+------+------+------+-----------+
|       428|     Sales|Sales Executive| 60|Female| 10266|    22|          1|
|       880|     Sales|Sales Executive| 60|  Male|  5220|    12|          2|
|       537|     Sales|Sales Executive| 60|  Male|  5405|    10|          3|
|        64|     Sales|Sales Executive| 59|Female|  7637|    28|          4|
|        71|     Sales|Sales Executive| 59|Female|  5473|    20|          5|
|       898|     Sales|Sales Executive| 59|Female|  5171|    13|          6|
|        99|     Sales|Sales Executive| 58|  Male| 13872|    38|          7|
|       402|     Sales|Sales Executive| 56|Female| 13212|    36|          8|
|      1371|     Sales|Sales Executive| 56|Female|  5380|     6|          9|
|        83|     Sales|Sales Executive| 55|  Male| 10239|    24|         10|

#### Lag()
      *Lag() it is window funciton that allows you access records from previous row.
      *It will be useful in compairing records between rows or calculating differences.
      *lag(column_name,offset,default value) - lag(income,2,0)

In [29]:
spark.sql("""
select employeeid,department,jobrole,age,gender,income,workex, 
lag(income) over(partition by jobrole order by employeeid) as previous_income
from hremployee
""").show()

+----------+----------+---------------+---+------+------+------+---------------+
|employeeid|department|        jobrole|age|gender|income|workex|previous_income|
+----------+----------+---------------+---+------+------+------+---------------+
|         1|     Sales|Sales Executive| 41|Female|  5993|     8|           null|
|      1012|     Sales|Sales Executive| 36|Female|  9278|    15|           5993|
|      1014|     Sales|Sales Executive| 30|Female|  4779|     8|           9278|
|      1020|     Sales|Sales Executive| 36|Female|  5647|    11|           4779|
|      1026|     Sales|Sales Executive| 24|Female|  4162|     5|           5647|
|      1027|     Sales|Sales Executive| 32|  Male|  9204|     7|           4162|
|      1031|     Sales|Sales Executive| 31|  Male| 10793|    13|           9204|
|      1032|     Sales|Sales Executive| 46|  Male| 10096|    28|          10793|
|      1039|     Sales|Sales Executive| 48|  Male|  5486|    15|          10096|
|      1042|     Sales|Sales

In [31]:
spark.sql("""
select employeeid,department,jobrole,age,gender,income,workex, 
lag(income,2,0) over(partition by jobrole order by employeeid) as previous_income
from hremployee
""").show()

+----------+----------+---------------+---+------+------+------+---------------+
|employeeid|department|        jobrole|age|gender|income|workex|previous_income|
+----------+----------+---------------+---+------+------+------+---------------+
|         1|     Sales|Sales Executive| 41|Female|  5993|     8|              0|
|      1012|     Sales|Sales Executive| 36|Female|  9278|    15|              0|
|      1014|     Sales|Sales Executive| 30|Female|  4779|     8|           5993|
|      1020|     Sales|Sales Executive| 36|Female|  5647|    11|           9278|
|      1026|     Sales|Sales Executive| 24|Female|  4162|     5|           4779|
|      1027|     Sales|Sales Executive| 32|  Male|  9204|     7|           5647|
|      1031|     Sales|Sales Executive| 31|  Male| 10793|    13|           4162|
|      1032|     Sales|Sales Executive| 46|  Male| 10096|    28|           9204|
|      1039|     Sales|Sales Executive| 48|  Male|  5486|    15|          10793|
|      1042|     Sales|Sales

#### Lead()

In [32]:
spark.sql("""
select employeeid,department,jobrole, age,gender,income,workex, lead(income,1,0)
over(partition by jobrole order by employeeid) as next_income
from hremployee
""").show()

+----------+----------+---------------+---+------+------+------+-----------+
|employeeid|department|        jobrole|age|gender|income|workex|next_income|
+----------+----------+---------------+---+------+------+------+-----------+
|         1|     Sales|Sales Executive| 41|Female|  5993|     8|       9278|
|      1012|     Sales|Sales Executive| 36|Female|  9278|    15|       4779|
|      1014|     Sales|Sales Executive| 30|Female|  4779|     8|       5647|
|      1020|     Sales|Sales Executive| 36|Female|  5647|    11|       4162|
|      1026|     Sales|Sales Executive| 24|Female|  4162|     5|       9204|
|      1027|     Sales|Sales Executive| 32|  Male|  9204|     7|      10793|
|      1031|     Sales|Sales Executive| 31|  Male| 10793|    13|      10096|
|      1032|     Sales|Sales Executive| 46|  Male| 10096|    28|       5486|
|      1039|     Sales|Sales Executive| 48|  Male|  5486|    15|       8463|
|      1042|     Sales|Sales Executive| 28|  Male|  8463|     6|       4373|

#### NTILE()
       * Dividing records into quantiles

In [37]:
spark.sql("""
select employeeid,department,jobrole, age,gender,income,workex, NTILE(4) 
OVER(order by income desc) as salary_quantile from hremployee
""").show(400)

+----------+--------------------+--------------------+---+------+------+------+---------------+
|employeeid|          department|             jobrole|age|gender|income|workex|salary_quantile|
+----------+--------------------+--------------------+---+------+------+------+---------------+
|       191|Research & Develo...|             Manager| 52|  Male| 19999|    34|              1|
|       747|Research & Develo...|   Research Director| 41|Female| 19973|    21|              1|
|       852|Research & Develo...|             Manager| 56|Female| 19943|    28|              1|
|       166|Research & Develo...|             Manager| 50|Female| 19926|    21|              1|
|       569|Research & Develo...|             Manager| 55|  Male| 19859|    24|              1|
|       919|               Sales|             Manager| 51|  Male| 19847|    31|              1|
|       750|               Sales|             Manager| 52|Female| 19845|    33|              1|
|      1242|               Sales|       

#### Calculate Max,Mean,Avg Income from each quantile

In [53]:
spark.sql("""
select Max(income) as Maximum, round(avg(income),2) as Mean, percentile_approx(income,0.5) as Median, salary_quantile from
(select employeeid,department,jobrole, age,gender,income,workex, NTILE(4)
OVER(order by income) as salary_quantile from hremployee) group by salary_quantile
""").show()

+-------+--------+------+---------------+
|Maximum|    Mean|Median|salary_quantile|
+-------+--------+------+---------------+
|   2911|  2354.0|  2377|              1|
|   4930| 3966.74|  4033|              2|
|   8380| 6191.14|  6091|              3|
|  19999|13522.33| 13120|              4|
+-------+--------+------+---------------+



#### Percent_Rank()
         * Window function that calculates relative rank() of row within a result set expressed as percentage.

In [54]:
spark.sql("""
select employeeid,department,jobrole, age,gender,income,workex, PERCENT_RANK() 
OVER(partition by department order by income DESC) as percent_rank from hremployee
""").show()

+----------+----------+-------+---+------+------+------+--------------------+
|employeeid|department|jobrole|age|gender|income|workex|        percent_rank|
+----------+----------+-------+---+------+------+------+--------------------+
|       919|     Sales|Manager| 51|  Male| 19847|    31|                 0.0|
|       750|     Sales|Manager| 52|Female| 19845|    33|0.002252252252252...|
|      1242|     Sales|Manager| 40|  Male| 19833|    21|0.004504504504504...|
|      1116|     Sales|Manager| 55|  Male| 19586|    36|0.006756756756756757|
|       234|     Sales|Manager| 50|Female| 19517|    32|0.009009009009009009|
|      1126|     Sales|Manager| 50|  Male| 19331|    27| 0.01126126126126126|
|       238|     Sales|Manager| 52|  Male| 19068|    33|0.013513513513513514|
|        30|     Sales|Manager| 46|Female| 18947|    22|0.015765765765765764|
|       914|     Sales|Manager| 45|Female| 18824|    26|0.018018018018018018|
|       917|     Sales|Manager| 46|Female| 18789|    26| 0.02027

#### Find out the number of records in each of the percentile range(0-25,25-50,50-75,75-100)

In [66]:
spark.sql("""
select Quantile_Group, count(*) from
(select employeeid,department,jobrole, age,gender,income,workex,
case((When percent_rank <=0.25 then 'q1') else,
     (when percent_rank <=0.50 then 'q2') else,
     (when percent_rank <=0.75 then 'q3') 
      else 'q4' end) as Quantile_Group PERCENT_RANK() 
OVER(order by income) as percent_rank from hremployee) as Quantile group by Quantile_Group
""").show()

ParseException: "\nmismatched input 'from' expecting <EOF>(line 2, pos 32)\n\n== SQL ==\n\nselect Quantile_Group, count(*) from\n--------------------------------^^^\n(select employeeid,department,jobrole, age,gender,income,workex,\ncase((When percent_rank <=0.25 then 'q1') else,\n     (when percent_rank <=0.50 then 'q2') else,\n     (when percent_rank <=0.75 then 'q3') \n      else 'q4' end) as Quantile_Group PERCENT_RANK() \nOVER(order by income) as percent_rank from hremployee) as Quantile group by Quantile_Group\n"

In [87]:
spark.sql("""
select Quantile_Group, count(*) as Quantile_Count from 
(
select *,case When percent_rank <=0.25 then 'q1'
     when percent_rank <=0.50 then 'q2'
     when percent_rank <=0.75 then 'q3'
      else 'q4' end as Quantile_Group 
from
(
select employeeid,department,jobrole, age,gender,income,workex,PERCENT_RANK() 
OVER(partition by department order by income) as percent_rank from hremployee
)) group by Quantile_Group order by Quantile_Count desc
""").show()

+--------------+--------------+
|Quantile_Group|Quantile_Count|
+--------------+--------------+
|            q1|           370|
|            q4|           367|
|            q2|           366|
|            q3|           366|
+--------------+--------------+



In [89]:
spark.sql("""
select Quantile_Group, count(*) from
(
select percent_rank,
case when percent_rank <=0.25 then 'Q1'
     when percent_rank <=0.50 then 'Q2'
     when percent_rank <=0.75 then 'Q3'
     else 'Q4' end as Quantile_Group 
from
(
select employeeid,department,jobrole, age,gender,income,workex,PERCENT_RANK() 
OVER(partition by department order by income) as percent_rank from hremployee)) group by Quantile_Group
""").show()

+--------------+--------+
|Quantile_Group|count(1)|
+--------------+--------+
|            Q2|     366|
|            Q1|     370|
|            Q3|     366|
|            Q4|     367|
+--------------+--------+



#### Show JobRole with maximum Attrition Rate (Percentage)

In [108]:
spark.sql("""
select jobrole, 
round(sum(case when Attrition = 'Yes' then 1
     else 0 end)/ count(EmployeeID)*100,2) as Percent from hremployee group by jobrole order by Percent desc
""").show()

+--------------------+-------+
|             jobrole|Percent|
+--------------------+-------+
|Sales Representative|  40.24|
|Laboratory Techni...|  23.94|
|     Human Resources|  23.08|
|     Sales Executive|  17.48|
|  Research Scientist|   16.1|
|Manufacturing Dir...|    6.9|
|Healthcare Repres...|   6.87|
|             Manager|    4.9|
|   Research Director|    2.5|
+--------------------+-------+



### HBase Intergation with Pyspark

In [1]:
spark.stop()

In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("HbaseIntegration").getOrCreate()

In [3]:
hbase_conf = {
    "hbase.zookeeper.quorum": 'localhost',
    "hbase.mapreduce.inputtable": 'hcustomer',
    "hbase.client.scanner.timeout.period": "600000"
}

In [ ]:
df = spark.read.format("org.apache.hadoop.hbase.spark").options(**hbase_conf).load()

### Hive Integration with pyspark

In [5]:
spark.stop()

In [6]:
spark = (SparkSession.builder.appName("pyspark-hive-integration")
         .config('spark.sql.warehouse.dir', '/user/hive/warehouse/')
         .enableHiveSupport().getOrCreate())

In [7]:
spark

In [8]:
spark.sql("show databases").show()

+------------+
|databaseName|
+------------+
|    airlines|
|  banking_db|
|     default|
+------------+



In [10]:
spark.sql("""
create table if not exists airlines.flights(DayOfMonth int, DayOfWeek int, Carrier varchar(10), OriginAirportID int, DestAirportID int, DepDelay int, ArrDelay int)
row format delimited
fields terminated by ','
lines terminated by '\n'
STORED AS TEXTFILE
tblProperties("skip.header.line.count" = 1)
""")

DataFrame[]

In [11]:
spark.sql("use airlines").show()

++
||
++
++



In [12]:
spark.sql("show tables").show()

+--------+---------+-----------+
|database|tableName|isTemporary|
+--------+---------+-----------+
|airlines|  flights|      false|
+--------+---------+-----------+



In [13]:
spark.sql("describe flights").show()

+---------------+---------+-------+
|       col_name|data_type|comment|
+---------------+---------+-------+
|     DayOfMonth|      int|   null|
|      DayOfWeek|      int|   null|
|        Carrier|   string|   null|
|OriginAirportID|      int|   null|
|  DestAirportID|      int|   null|
|       DepDelay|      int|   null|
|       ArrDelay|      int|   null|
+---------------+---------+-------+



In [17]:
spark.sql("""
load data local inpath '/home/hadoop/Downloads/raw_flight_data1.csv' 
overwrite into table flights
""")

DataFrame[]

In [19]:
spark.sql("select * from flights").show()

+----------+---------+-------+---------------+-------------+--------+--------+
|DayOfMonth|DayOfWeek|Carrier|OriginAirportID|DestAirportID|DepDelay|ArrDelay|
+----------+---------+-------+---------------+-------------+--------+--------+
|        19|        5|     DL|          11433|        13303|      -3|       1|
|        19|        5|     DL|          14869|        12478|       0|      -8|
|        19|        5|     DL|          14057|        14869|      -4|     -15|
|        19|        5|     DL|          15016|        11433|      28|      24|
|        19|        5|     DL|          11193|        12892|      -6|     -11|
|        19|        5|     DL|          10397|        15016|      -1|     -19|
|        19|        5|     DL|          15016|        10397|       0|      -1|
|        19|        5|     DL|          10397|        14869|      15|      24|
|        19|        5|     DL|          10397|        10423|      33|      34|
|        19|        5|     DL|          11278|      

In [20]:
spark.sql("""
create table if not exists airlines.airline(airport_id int,city varchar(30),state varchar(10),name varchar(70))
row format delimited
fields terminated by ','
lines terminated by '\n'
STORED AS TEXTFILE
tblProperties("skip.header.line.count" = 1)
""")

DataFrame[]

In [21]:
spark.sql("show tables").show()

+--------+---------+-----------+
|database|tableName|isTemporary|
+--------+---------+-----------+
|airlines|  airline|      false|
|airlines|  flights|      false|
+--------+---------+-----------+



In [22]:
spark.sql("""
load data local inpath '/home/hadoop/Downloads/airports1.csv' 
overwrite into table airline
""")

DataFrame[]

In [24]:
spark.sql("describe airline").show()

+----------+---------+-------+
|  col_name|data_type|comment|
+----------+---------+-------+
|airport_id|      int|   null|
|      city|   string|   null|
|     state|   string|   null|
|      name|   string|   null|
+----------+---------+-------+



In [25]:
spark.sql("select * from airline limit 10").show()

+----------+-----------+-----+--------------------+
|airport_id|       city|state|                name|
+----------+-----------+-----+--------------------+
|     10165|Adak Island|   AK|                Adak|
|     10299|  Anchorage|   AK|Ted Stevens Ancho...|
|     10304|      Aniak|   AK|       Aniak Airport|
|     10754|     Barrow|   AK|Wiley Post/Will R...|
|     10551|     Bethel|   AK|      Bethel Airport|
|     10926|    Cordova|   AK|Merle K Mudhole S...|
|     14709|  Deadhorse|   AK|   Deadhorse Airport|
|     11336| Dillingham|   AK|  Dillingham Airport|
|     11630|  Fairbanks|   AK|Fairbanks Interna...|
|     11997|   Gustavus|   AK|    Gustavus Airport|
+----------+-----------+-----+--------------------+



In [26]:
flights_df = spark.table("flights")
airports_df = spark.table("airline")

In [27]:
flights_df.show(2)

+----------+---------+-------+---------------+-------------+--------+--------+
|DayOfMonth|DayOfWeek|Carrier|OriginAirportID|DestAirportID|DepDelay|ArrDelay|
+----------+---------+-------+---------------+-------------+--------+--------+
|        19|        5|     DL|          11433|        13303|      -3|       1|
|        19|        5|     DL|          14869|        12478|       0|      -8|
+----------+---------+-------+---------------+-------------+--------+--------+
only showing top 2 rows



In [30]:
airports_df.show(2)

+----------+-----------+-----+--------------------+
|airport_id|       city|state|                name|
+----------+-----------+-----+--------------------+
|     10165|Adak Island|   AK|                Adak|
|     10299|  Anchorage|   AK|Ted Stevens Ancho...|
+----------+-----------+-----+--------------------+
only showing top 2 rows



### Transformation

In [35]:
flights_join = flights_df.join(airports_df, on=airports_df.airport_id == flights_df.OriginAirportID, how='inner')

In [36]:
flights_join.show(2)

+----------+---------+-------+---------------+-------------+--------+--------+----------+--------------+-----+--------------------+
|DayOfMonth|DayOfWeek|Carrier|OriginAirportID|DestAirportID|DepDelay|ArrDelay|airport_id|          city|state|                name|
+----------+---------+-------+---------------+-------------+--------+--------+----------+--------------+-----+--------------------+
|        19|        5|     DL|          11433|        13303|      -3|       1|     11433|       Detroit|   MI|Detroit Metro Way...|
|        19|        5|     DL|          14869|        12478|       0|      -8|     14869|Salt Lake City|   UT|Salt Lake City In...|
+----------+---------+-------+---------------+-------------+--------+--------+----------+--------------+-----+--------------------+
only showing top 2 rows



In [37]:
flights_ljoin = flights_df.join(airports_df, on=airports_df.airport_id == flights_df.OriginAirportID, how='left')
flights_ljoin.show(2)

+----------+---------+-------+---------------+-------------+--------+--------+----------+--------------+-----+--------------------+
|DayOfMonth|DayOfWeek|Carrier|OriginAirportID|DestAirportID|DepDelay|ArrDelay|airport_id|          city|state|                name|
+----------+---------+-------+---------------+-------------+--------+--------+----------+--------------+-----+--------------------+
|        19|        5|     DL|          11433|        13303|      -3|       1|     11433|       Detroit|   MI|Detroit Metro Way...|
|        19|        5|     DL|          14869|        12478|       0|      -8|     14869|Salt Lake City|   UT|Salt Lake City In...|
+----------+---------+-------+---------------+-------------+--------+--------+----------+--------------+-----+--------------------+
only showing top 2 rows



In [38]:
flights_rjoin = flights_df.join(airports_df, on=airports_df.airport_id == flights_df.OriginAirportID, how='right')
flights_rjoin.show(2)

+----------+---------+-------+---------------+-------------+--------+--------+----------+----+-----+--------------------+
|DayOfMonth|DayOfWeek|Carrier|OriginAirportID|DestAirportID|DepDelay|ArrDelay|airport_id|city|state|                name|
+----------+---------+-------+---------------+-------------+--------+--------+----------+----+-----+--------------------+
|        19|        5|     DL|          14570|        14869|      -5|      -8|     14570|Reno|   NV|Reno/Tahoe Intern...|
|        19|        5|     DL|          14570|        14869|      -4|       2|     14570|Reno|   NV|Reno/Tahoe Intern...|
+----------+---------+-------+---------------+-------------+--------+--------+----------+----+-----+--------------------+
only showing top 2 rows



In [39]:
flights_ojoin = flights_df.join(airports_df, on=airports_df.airport_id == flights_df.OriginAirportID, how='outer')
flights_ojoin.show(2)

+----------+---------+-------+---------------+-------------+--------+--------+----------+----+-----+--------------------+
|DayOfMonth|DayOfWeek|Carrier|OriginAirportID|DestAirportID|DepDelay|ArrDelay|airport_id|city|state|                name|
+----------+---------+-------+---------------+-------------+--------+--------+----------+----+-----+--------------------+
|        19|        5|     DL|          14570|        14869|      -5|      -8|     14570|Reno|   NV|Reno/Tahoe Intern...|
|        19|        5|     DL|          14570|        14869|      -4|       2|     14570|Reno|   NV|Reno/Tahoe Intern...|
+----------+---------+-------+---------------+-------------+--------+--------+----------+----+-----+--------------------+
only showing top 2 rows



In [40]:
flights_ltjoin = flights_df.join(airports_df, on=airports_df.airport_id == flights_df.OriginAirportID, how='leftanti')
flights_ltjoin.show(2)

+----------+---------+-------+---------------+-------------+--------+--------+
|DayOfMonth|DayOfWeek|Carrier|OriginAirportID|DestAirportID|DepDelay|ArrDelay|
+----------+---------+-------+---------------+-------------+--------+--------+
+----------+---------+-------+---------------+-------------+--------+--------+



In [42]:
flights_lsjoin = flights_df.join(airports_df, on=airports_df.airport_id == flights_df.OriginAirportID, how='leftsemi')
flights_lsjoin.show(2)

+----------+---------+-------+---------------+-------------+--------+--------+
|DayOfMonth|DayOfWeek|Carrier|OriginAirportID|DestAirportID|DepDelay|ArrDelay|
+----------+---------+-------+---------------+-------------+--------+--------+
|        19|        5|     DL|          11433|        13303|      -3|       1|
|        19|        5|     DL|          14869|        12478|       0|      -8|
+----------+---------+-------+---------------+-------------+--------+--------+
only showing top 2 rows



In [ ]:
flights_join.repartition()

In [43]:
flights_join.write.partitionBy('Carrier').parquet("/airlines")

In [44]:
airlines_join = spark.read.parquet("/airlines")

In [45]:
airlines_join.createOrReplaceTempView('airlines')

In [47]:
spark.sql("select avg(ArrDelay) as Arrival_Delay from airlines where Carrier = 'AA'").show()

+------------------+
|     Arrival_Delay|
+------------------+
|11.651327332054372|
+------------------+



In [48]:
airlines_join.printSchema()

root
 |-- DayOfMonth: integer (nullable = true)
 |-- DayOfWeek: integer (nullable = true)
 |-- OriginAirportID: integer (nullable = true)
 |-- DestAirportID: integer (nullable = true)
 |-- DepDelay: integer (nullable = true)
 |-- ArrDelay: integer (nullable = true)
 |-- airport_id: integer (nullable = true)
 |-- city: string (nullable = true)
 |-- state: string (nullable = true)
 |-- name: string (nullable = true)
 |-- Carrier: string (nullable = true)



### Load on Mysql

In [49]:
connection_properties = {
    "user":"root",
    "password":"hadoop@123",
    "driver":"com.mysql.cj.jdbc.Driver"
}
airlines_join.write.jdbc(url = "jdbc:mysql://localhost:3306/airlines",table = "airport",
                 mode = "overwrite", properties = connection_properties)